<a href="https://colab.research.google.com/github/Garvit1721/facia_image_recognition/blob/main/facial_image_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 12.69 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [ ]:
import csv
data = {}
with open('/content/facial_expressions/data/legend.csv') as f:
  reader = csv.reader(f)
  next(reader)
  for row in reader :
    key = row[2].lower()
    if key in data :
      data[key].append(row[1])
    else :
      data[key] = [row[1]]

In [ ]:
emotion_list = list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [ ]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [ ]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training',emotion))
  os.mkdir(os.path.join('master_data/testing',emotion))

In [ ]:
import os
import shutil

split_size = 0.8

for emotion, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]

    for image in train_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/training', emotion, os.path.basename(image))
        shutil.copyfile(source, dest)

    for image in test_images:
        source = os.path.join('/content/facial_expressions/images', image)
        dest = os.path.join('/content/master_data/testing', emotion, os.path.basename(image))
        shutil.copyfile(source, dest)


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten
from keras.models import Sequential

In [ ]:
model = Sequential()
layer1 = Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3))
model.add(layer1)
layer2 = MaxPooling2D(2,2)
model.add(layer2)
layer3 = Conv2D(32,(3,3),activation='relu')
model.add(layer3)
layer = MaxPooling2D(2,2)
model.add(layer)
layer4 = Conv2D(64,(3,3),activation='relu')
model.add(layer4)
layer5 = MaxPooling2D(2,2)
model.add(layer5)
layer6 = Flatten()
model.add(layer6)
layer7 = Dense(512,activation='relu')
model.add(layer7)
layer8 = Dense(8,activation='softmax')
model.add(layer8)

In [ ]:
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 49, 49, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 10, 10, 64)       

In [ ]:
train_dir = '/content/master_data/training'
test_dir = '/content/master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(100,100),
    class_mode='categorical',
    batch_size=128
)

Found 10941 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [ ]:
es = EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [ ]:
model.fit_generator(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=test_generator,
    callbacks=[es]
)

<ipython-input-19-c75d829bc7ea>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
86/86 [==============================] - 19s 220ms/step - loss: 0.2241 - acc: 0.9185 - val_loss: 2.0648 - val_acc: 0.6674
Epoch 2/10
86/86 [==============================] - 16s 191ms/step - loss: 0.1782 - acc: 0.9350 - val_loss: 2.6998 - val_acc: 0.6543
Epoch 3/10
86/86 [==============================] - 16s 191ms/step - loss: 0.1607 - acc: 0.9400 - val_loss: 2.7889 - val_acc: 0.6670
